<a href="https://colab.research.google.com/github/aknip/Modal/blob/main/runpod_api_endpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dev in Notebook, Deployment to RunPod

This notebook shows

Code blocks wich are needed for the final .py-file (for Modal.com) are marked with `#|export`

Sources:  
- https://blog.runpod.io/serverless-create-a-basic-api/


In [1]:
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])

In [16]:
%%capture --no-stderr
!pip install runpod docker -q

# Part 1: RunPod App (init)



In [ ]:
# %%writefile whatever.py
import runpod

def is_even(job):

    job_input = job["input"]
    the_number = job_input["number"]

    if not isinstance(the_number, int):
        return {"error": "Silly human, you need to pass an integer."}

    if the_number % 2 == 0:
        return True

    return False

runpod.serverless.start({"handler": is_even})

In [11]:
print(is_even({"input": {"number": 2}}))


True


In [4]:
# %%writefile test_input.json
{
    "input": {
        "number": 2
    }
}

Writing test_input.json


In [14]:
!python3 whatever.py

--- Starting Serverless Worker |  Version 1.3.3 ---
INFO   | Using test_input.json as job input.
DEBUG  | Retrieved local job: {'input': {'number': 2}, 'id': 'local_test'}
INFO   | local_test | Started
DEBUG  | local_test | Handler output: True
DEBUG  | local_test | run_job return: {'output': True}
INFO   | Job local_test completed successfully.
INFO   | Job result: {'output': True}
INFO   | Local testing complete, exiting.


In [15]:
# %%writefile Dockerfile
# Include Python
from python:3.11.1-buster

# Define your working directory
WORKDIR /

# Install runpod
RUN pip install runpod

# Add your file
ADD whatever.py .

# Call your file when your container starts
CMD [ "python", "-u", "/whatever.py" ]

Writing Dockerfile


In [17]:
!docker build . --tag=repo/name

/bin/bash: line 1: docker: command not found


In [ ]:
#|export



# Export production file for Modal with nbdev

The notebook file will be copied from Google drive to the current notebook environment.

Alternative?: %notebook modal_hello_world2.ipynb

In [ ]:
# Connect Google Drive
# This and the following cell can be skipped it the notebook file is copied manually to the current notebook environment
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Copy Notebook to local path
!cp "/content/drive/MyDrive/Colab Notebooks/modal-gradio.ipynb" /content/

In [ ]:
# Export source code marked with #|export
from nbdev.export import nb_export
nb_export('/content/modal-gradio.ipynb', lib_path='.', name='gradio_modal')

# Run in Modal

In [ ]:
# dev server
!modal serve gradio_modal.py

In [ ]:
# Deploy server permanently
!modal deploy gradio_modal.py

# Idea: Modify exported code if necessary

In [ ]:
# Optimize exported source code
# Creates a backup file automatically
import io
import os
from datetime import datetime

# load
f= open('example.py','r')
if f.mode == 'r':
      source_code =f.read()
      #print(source_code)
f.close()

# modifiy code
source_code = source_code.replace("#stub", "stub")
source_code = source_code.replace("#@stub", "@stub")
source_code = source_code + "\nTest"

# save
current_date_time = datetime.now().strftime("_%Y-%m-%d_%H-%M-%S")
os.rename('example.py', 'example' + current_date_time + '.py')
f= open('example.py','w+')
f.write(source_code)
f.close()